# Предсказание стоимости жилья в Калифорнии

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Пропуски-в-данных" data-toc-modified-id="Пропуски-в-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Пропуски в данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li></ul></li><li><span><a href="#Кореляция-с-целевым-признаком" data-toc-modified-id="Кореляция-с-целевым-признаком-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Кореляция с целевым признаком</a></span></li></ul></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка признаков</a></span><ul class="toc-item"><li><span><a href="#Разделение-на-выборки" data-toc-modified-id="Разделение-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение на выборки</a></span></li><li><span><a href="#Pipeline-для-подготовки-признаков" data-toc-modified-id="Pipeline-для-подготовки-признаков-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Pipeline для подготовки признаков</a></span></li></ul></li><li><span><a href="#Модели" data-toc-modified-id="Модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Модели</a></span><ul class="toc-item"><li><span><a href="#Обучение-LinearRegression-на-всех-данных" data-toc-modified-id="Обучение-LinearRegression-на-всех-данных-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Обучение LinearRegression на всех данных</a></span></li><li><span><a href="#Обучение-LinearRegression-на-числовых-данных" data-toc-modified-id="Обучение-LinearRegression-на-числовых-данных-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Обучение LinearRegression на числовых данных</a></span></li><li><span><a href="#Метрики-качества-моделей" data-toc-modified-id="Метрики-качества-моделей-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Метрики качества моделей</a></span><ul class="toc-item"><li><span><a href="#Сравнение-метрик-качества-моделей" data-toc-modified-id="Сравнение-метрик-качества-моделей-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Сравнение метрик качества моделей</a></span></li></ul></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

<div style="padding: 30px 25px; border: 2px #6495ed solid">

__Цель:__ Используя оценщик LinearRegression из библиотеки MLlib реализовать две модели, построенные на различных наборах данных:
- Используя числовые и категориальные данные;
- Используя только числовые данные, исключив категориальные.

__Оценка результата:__ Сравнение качества работы моделей, по метрикам:
- RMSE;
- MAE;
- R2.
    

    
<br>__Данные:__ Информация о жилье в Калифорнии в 1990 году.</font>
<details>
<summary><u>Описание данных</u></summary>


__В колонках датасета содержатся следующие данные:__

- longitude — широта;
- latitude — долгота;
- housing_median_age — медианный возраст жителей жилого массива;
- total_rooms — общее количество комнат в домах жилого массива;
- total_bedrooms — общее количество спален в домах жилого массива;
- population — количество человек, которые проживают в жилом массиве;
- households — количество домовладений в жилом массиве;
- median_income — медианный доход жителей жилого массива;
- median_house_value — медианная стоимость дома в жилом массиве;
- ocean_proximity — близость к океану.

---    

__Целевой признак: median_house_value__
</details>    
</br>
</div>    

In [3]:
# библиотеки используемые в проекте
import pandas as pd 
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
 
from pyspark.sql.functions import isnan, isnull, count, col, sum, when

# константы
RANDOM_SEED = 42

In [4]:
# запуск spark-сессии
spark = SparkSession.builder \
                    .master('local') \
                    .appName('spark_home_value_prediction') \
                    .getOrCreate()

## Подготовка данных
### Загрузка данных

In [5]:
# загрузка данных из файла 'housing.csv'
df = spark.read.csv('housing.csv', header=True, inferSchema=True)

In [7]:
# Подсчет количества строк в DataFrame
row_count = df.count()

display(f'Количество строк в DataFrame: {row_count}')

'Количество строк в DataFrame: 20640'

In [8]:
# вывод типов данных колонок DataFrame
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [10]:
# вывод строк DataFrame
df.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 2 rows



In [11]:
# общая статистика по столбцам
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
longitude,20640,-119.56970445736148,2.003531723502584,-124.35,-114.31
latitude,20640,35.6318614341087,2.135952397457101,32.54,41.95
housing_median_age,20640,28.639486434108527,12.58555761211163,1.0,52.0
total_rooms,20640,2635.7630813953488,2181.6152515827944,2.0,39320.0
total_bedrooms,20433,537.8705525375618,421.38507007403115,1.0,6445.0
population,20640,1425.4767441860465,1132.46212176534,3.0,35682.0
households,20640,499.5396802325581,382.3297528316098,1.0,6082.0
median_income,20640,3.8706710029070246,1.899821717945263,0.4999,15.0001
median_house_value,20640,206855.81690891474,115395.61587441359,14999.0,500001.0


In [12]:
# преобразование в цельночисленный формат столбцы 'total_rooms', 'total_bedrooms', 'population', 'households'
columns_to_convert = ['total_rooms', 'total_bedrooms', 'population', 'households']

for column in columns_to_convert:
    df = df.withColumn(column, F.col(column).cast('int'))

In [12]:
# выделим числовые, текстовые и целевой признак
categorical_cols = ['ocean_proximity']
numerical_cols  = ['median_income', 'households', 'population', 'total_bedrooms', 'total_rooms',
                  'housing_median_age', 'longitude', 'latitude']
target = 'median_house_value' 

### Пропуски в данных

In [14]:
# выведим количество пропусков в столбцах
null = [(column,df.filter(F.col(column).isNull()).count()) for column in df.columns]
for column, count in null:
    display(f'{column}: {count}')

'longitude: 0'

'latitude: 0'

'housing_median_age: 0'

'total_rooms: 0'

'total_bedrooms: 207'

'population: 0'

'households: 0'

'median_income: 0'

'median_house_value: 0'

'ocean_proximity: 0'

<div style="padding: 30px 25px; border: 2px #6495ed solid">
Пропуски присутствуют в столбце 'total bedrooms' - общее количество спален в домах жилого массива;
</div>

#### Обработка пропусков

In [15]:
# корреляция с total bedrooms остальных признаков
for i in numerical_cols:
    print(f"Correlation to total_bedrooms for {i}={df.stat.corr('total_bedrooms', i)}")

Correlation to total_bedrooms for median_income=-0.0072945417081858544
Correlation to total_bedrooms for households=0.966507240042043
Correlation to total_bedrooms for population=0.8662661985860806
Correlation to total_bedrooms for total_bedrooms=1.0
Correlation to total_bedrooms for total_rooms=0.9201961721166215
Correlation to total_bedrooms for housing_median_age=-0.31706334936263136
Correlation to total_bedrooms for longitude=0.06808179725677305
Correlation to total_bedrooms for latitude=-0.06531831669569808


<div style="padding: 30px 25px; border: 2px #6495ed solid">
Наибольшей линейной связью 'total bedrooms' обладает с households - количество домовладений в жилом массиве.
    </div>

In [16]:
# расчет количества спален на домовладение
df = df.withColumn('bedrooms_per_household', F.col('total_bedrooms')/F.col('households'))

In [17]:
# среднее количество спален на дом
mean_value = round(df.select(F.mean('bedrooms_per_household')).first()[0])
mean_value

1

<div style="padding: 30px 25px; border: 2px #6495ed solid">
так как средне значение спален на домовладение равно 1, соответственно количество спален будет равно количеству домовладений в жилом массиве.
    </div>

In [18]:
# заполнение пропусков в столбце total_bedrooms значениями из households
df = df.withColumn("total_bedrooms", when(df["total_bedrooms"].isNull(), df['households']).otherwise(df["total_bedrooms"]))

In [19]:
# удаление вспомогательного столбца 'bedrooms_per_household'
df = df.drop(F.col('bedrooms_per_household'))

In [20]:
# выведим количество пропусков в столбцах
null = [(column,df.filter(F.col(column).isNull()).count()) for column in df.columns]
for column, count in null:
    display(f'{column}: {count}')

'longitude: 0'

'latitude: 0'

'housing_median_age: 0'

'total_rooms: 0'

'total_bedrooms: 0'

'population: 0'

'households: 0'

'median_income: 0'

'median_house_value: 0'

'ocean_proximity: 0'

<div style="padding: 30px 25px; border: 2px #6495ed solid">
Пропуски заполнили
    </div>

### Кореляция с целевым признаком

In [21]:
# корреляция с total bedrooms остальных признаков
for i in numerical_cols:
    print(f"Correlation to {target} for {i}={df.stat.corr(target, i)}")

Correlation to median_house_value for median_income=0.6880752079585578
Correlation to median_house_value for households=0.06584265057005637
Correlation to median_house_value for population=-0.024649678888894876
Correlation to median_house_value for total_bedrooms=0.050688452962582564
Correlation to median_house_value for total_rooms=0.13415311380656275
Correlation to median_house_value for housing_median_age=0.10562341249321067
Correlation to median_house_value for longitude=-0.045966615117981745
Correlation to median_house_value for latitude=-0.14416027687465752


<div style="padding: 30px 25px; border: 2px #6495ed solid">
Наиболишая линейная зависимость стоимости жилья наблюдается с 'median income' - медианный доход жителей жилого массива
    </div>

## Подготовка признаков

In [21]:
# переименуем целевой признак в 'label' 
df = df.withColumnRenamed(target, 'label')

### Разделение на выборки
Разделим датасет в следующих пропорциях:
- train_data 80%
- test_data 20%

In [40]:
train_data, test_data = df.randomSplit([.8,.2], seed=RANDOM_SEED)

display(f'{train_data.count()/df.count():.0%}, {test_data.count()/df.count():.0%}') 

'80%, 20%'

### Pipeline для подготовки признаков

In [41]:
# Pipeline для категориальных и числовых признаков
stages = []
# трансформируем категориальные признаки
# трансформер StringIndexer
indexer = StringIndexer(inputCols=categorical_cols, 
                        outputCols=[c+'_idx' for c in categorical_cols],
                        handleInvalid = 'keep')
stages += [indexer]
# OHE-кодирование
encoder = OneHotEncoder(inputCols=[c+'_idx' for c in categorical_cols],
                        outputCols=[c+'_ohe' for c in categorical_cols])
stages += [encoder]
# объединение категориальных признаков в один вектор
categorical_assembler = \
        VectorAssembler(inputCols=[c+'_ohe' for c in categorical_cols],
                        outputCol="categorical_features")
stages += [categorical_assembler]

# трансформируем числовые признаки
# шкалирование значений 
numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                      outputCol="numerical_features")
stages += [numerical_assembler]
#StandardScaler
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol="numerical_features_scaled")
stages += [standardScaler]
# объединение признаков в один вектор
all_features = ['categorical_features','numerical_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol='features')
stages += [final_assembler]
# модель
lr = LinearRegression(labelCol='label', featuresCol='features')
stages += [lr]
# задаем план stages для обучения модели 
pipeline = Pipeline(stages=stages)

In [42]:
# Pipeline для числовых признаков
stages_num = []
# трансформируем числовые признаки
# шкалирование значений 
numerical_assembler = VectorAssembler(inputCols=numerical_cols,
                                      outputCol='numerical_features')
stages_num += [numerical_assembler]
#StandardScaler
standardScaler = StandardScaler(inputCol='numerical_features',
                                outputCol='numerical_features_scaled')
stages_num += [standardScaler]

# модель
lr_num = LinearRegression(labelCol='label', featuresCol='numerical_features_scaled')
stages_num += [lr_num]
# задаем план stages_num для обучения модели 
pipeline_num = Pipeline(stages=stages_num)

## Модели

При построении моделей используем подбор по сетке гиперпараметров с кросс-валидацией

In [43]:
# сетка гиперпараметров
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, 0.01, 0.1]) \
    .addGrid(lr.elasticNetParam, [0.5, 1.0]) \
    .build()

# кросс-валидатор для всего набора признаков
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=RegressionEvaluator(metricName='rmse'),
                    numFolds=4)
# кросс-валидатор для числовых признаков
cv_num = CrossValidator(estimator=pipeline_num,
                    estimatorParamMaps=paramGrid,
                    evaluator=RegressionEvaluator(metricName='rmse'),
                    numFolds=4)


### Обучение LinearRegression на всех данных

In [ ]:
# обучаем модель
cv_model_all = cv.fit(train_data)

In [45]:
# создаем предикт
predictions_all = cv_model_all.transform(test_data)
predictedLabes_all = predictions_all.select('label', 'prediction')
predictedLabes_all.show(3) 

+--------+------------------+
|   label|        prediction|
+--------+------------------+
|103600.0|150354.36048713746|
|106700.0|216870.72990827635|
| 73200.0|125983.70578138065|
+--------+------------------+
only showing top 3 rows



### Обучение LinearRegression на числовых данных

In [ ]:
# обучаем модель
cv_model_num = cv_num.fit(train_data)

In [47]:
# создаем предикт
predictions_num = cv_model_num.transform(test_data)
predictedLabes_num = predictions_num.select('label', 'prediction')
predictedLabes_num.show(3) 

+--------+------------------+
|   label|        prediction|
+--------+------------------+
|103600.0|101389.51858995808|
|106700.0|190033.05866057798|
| 73200.0| 76417.54363664845|
+--------+------------------+
only showing top 3 rows



### Метрики качества моделей

Сравним результаты работы линейной регрессии на двух наборах данных по метрикам RMSE, MAE и R2

In [48]:
# рассчет RMSE
evaluator_rmse = RegressionEvaluator(predictionCol="prediction", labelCol='label', metricName="rmse")

rmse_all = evaluator_rmse.evaluate(predictedLabes_all)
rmse_num = evaluator_rmse.evaluate(predictedLabes_num)

In [49]:
# рассчет MAE
evaluator_mae = RegressionEvaluator(predictionCol="prediction", labelCol='label', metricName="mae")

mae_all = evaluator_mae.evaluate(predictedLabes_all)
mae_num = evaluator_mae.evaluate(predictedLabes_num)

In [50]:
# рассчет R2
evaluator_r2 = RegressionEvaluator(predictionCol="prediction", labelCol='label', metricName="r2")

r2_all = evaluator_r2.evaluate(predictedLabes_all)
r2_num = evaluator_r2.evaluate(predictedLabes_num)

#### Сравнение метрик качества моделей

In [52]:
# сводная таблица метрик
info=pd.DataFrame({'Метрики':['RMSE', 'MAE', 'R2'],
                      'LR по всем данным':[rmse_all, mae_all, r2_all],
                  'LR по числовым данным':[rmse_num, mae_num, r2_num]})
display(info)

,Метрики,LR по всем данным,LR по числовым данным
0,RMSE,70610.236723,71567.036768
1,MAE,50783.403910,51675.146316
2,R2,0.639646,0.629814


In [ ]:
# закрытие spark-сессии
spark.stop()

## Вывод

<div style="padding: 30px 25px; border: 2px #6495ed solid">
    
- Для предобработки данных был применен алгоритм, реализованный в рамках фреймворка Spark.
- Для создания двух моделей был использован оценщик LinearRegression из библиотеки MLlib, и каждая модель была построена на отдельном наборе данных.
- Было проведено сравнение метрик качества моделей для определения их относительной эффективности.    
- Использование как числовых, так и категориальных признаков в модели приводит к улучшению ее качества по всем использованным метрикам качества, в сравнении с моделью, обученной только на числовых данных.
- Полученные значения итоговых метрик являются очень низкими. Для улучшения этих метрик стоит рассмотреть возможность применения других алгоритмов обучения.
</div>